In [8]:
# 필요한 라이브러리 및 모듈 임포트

import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from efficientnet_pytorch import EfficientNet
torch.cuda.empty_cache()


In [9]:
if torch.cuda.is_available():
    print("CUDA 사용 가능")
    print("사용 중인 CUDA 장치 개수:", torch.cuda.device_count())
    print("현재 사용 중인 CUDA 장치:", torch.cuda.current_device())
else:
    print("CUDA 사용 불가능")

CUDA 사용 가능
사용 중인 CUDA 장치 개수: 1
현재 사용 중인 CUDA 장치: 0


In [10]:
hyper_param_batch = 2

# 난수 시드 설정
random_seed = 100
random.seed(random_seed)
torch.manual_seed(random_seed)

# 클래스 개수 및 사용할 모델 이름 설정
num_classes = 4
model_name = 'efficientnet-b7'

train_name = 'model4'

# 모델 및 데이터 경로 설정
PATH = '/aiffel/aiffel/team_project/model4/scalp_weights/'

# data_train_path = './train_data/'+train_name+'/train'
# data_validation_path = './train_data/'+train_name+'/validation'

data_train_path = '/aiffel/aiffel/team_project/'+train_name+'/train'
data_validation_path = '/aiffel/aiffel/team_project/'+train_name+'/validation'
data_test_path = '/aiffel/aiffel/team_project/'+train_name+'/test'

# EfficientNet 모델의 입력 이미지 크기 확인 및 출력
image_size = EfficientNet.get_image_size(model_name)
print(image_size)

# 미리 학습된 EfficientNet 모델 로드
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes)
model = model.to('cuda')



600
Loaded pretrained weights for efficientnet-b7


In [11]:

# 데이터 전처리를 위한 변환 정의
transforms_train = transforms.Compose([
    transforms.Resize([int(600), int(600)], interpolation=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Lambda(lambda x: x.rotate(90)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transforms_val = transforms.Compose([
    transforms.Resize([int(600), int(600)], interpolation=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def is_image_file(filename):
    return filename.lower().endswith(('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp'))

def is_valid_folder(foldername):
    return foldername != '.ipynb_checkpoints'

# 데이터셋 로드
train_data_set = datasets.ImageFolder(data_train_path, transform=transforms_train)
val_data_set = datasets.ImageFolder(data_validation_path, transform=transforms_val, is_valid_file=is_image_file)


# 데이터 로더 설정
dataloaders, batch_num = {}, {}
dataloaders['train'] = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True, num_workers=1)
dataloaders['val'] = DataLoader(val_data_set, batch_size=hyper_param_batch, shuffle=False, num_workers=1)
batch_num['train'], batch_num['val'] = len(train_data_set), len(val_data_set)

# 배치 크기 및 데이터셋 크기 출력
print('batch_size : %d,  train/val : %d / %d' % (hyper_param_batch, batch_num['train'], batch_num['val']))

# 클래스 이름 출력
class_names = train_data_set.classes
print(class_names)


batch_size : 2,  train/val : 3750 / 1070
['0', '1', '2', '3']


In [12]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.best_model_wts = None  # 최적의 모델 가중치 저장 변수

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model_wts = copy.deepcopy(model.state_dict())
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_wts = copy.deepcopy(model.state_dict())
            self.counter = 0

    # 모델 저장은 train_model 함수에서 처리

# train_model 함수의 코드는 상기 설명에 따라 수정


In [13]:
def train_model(model, criterion, optimizer, scheduler, early_stopping, num_epochs=25):
    start_time = time.time()
    since = time.time()
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    train_loss, train_acc, val_loss, val_acc = [], [], [], []

    for epoch in range(num_epochs):
        if early_stopping.early_stop:
            print("Early stopping")
            break

        if len(val_loss) != 0:
            best_val_loss = min(val_loss)
            early_stopping(best_val_loss, model)

        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        epoch_start = time.time()

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            num_cnt = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to('cuda')
                labels = labels.to('cuda')

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)
                
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                val_loss.append(epoch_loss)
                val_acc.append(epoch_acc)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                
            if phase == 'val' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('==> best model saved - %d / %.1f'%(best_idx, best_acc))
                
            epoch_end = time.time() - epoch_start
            print('Training epochs {} in {:.0f}m {:.0f}s'.format(epoch, epoch_end // 60, epoch_end % 60))
            print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))

    model.load_state_dict(best_model_wts)

    torch.save(model, PATH + 'aram_'+train_name+'.pt')
    torch.save(model.state_dict(), PATH + 'president_aram_'+train_name+'.pt')
    print('model saved')

    end_sec = time.time() - start_time
    end_times = str(datetime.timedelta(seconds=end_sec)).split('.')
    end_time = end_times[0]
    print("end time :", end_time)
    
    return model, best_idx, best_acc, train_loss, train_acc, val_loss, val_acc


In [ ]:

# 손실 함수 설정
criterion = nn.CrossEntropyLoss()

# 옵티마이저 설정
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

# 학습률 스케줄러 설정
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
early_stopping = EarlyStopping(patience=3, verbose=True)
# 학습 수행 횟수 설정 및 학습 함수 호출
num_epochs = 30
train_model(model, criterion, optimizer_ft, ex
            p_lr_scheduler,early_stopping, num_epochs=num_epochs)


Epoch 0/29
----------
train Loss: 0.9389 Acc: 62.0267
Training epochs 0 in 22m 24s

val Loss: 0.9564 Acc: 56.1682
==> best model saved - 0 / 56.2
Training epochs 0 in 24m 4s

Epoch 1/29
----------
train Loss: 0.7955 Acc: 67.3867
Training epochs 1 in 22m 24s

val Loss: 0.7720 Acc: 72.1495
==> best model saved - 1 / 72.1
Training epochs 1 in 24m 4s

Epoch 2/29
----------
val Loss: 0.8515 Acc: 71.7757
Training epochs 2 in 24m 5s

Epoch 3/29
----------
train Loss: 0.6859 Acc: 71.6267
Training epochs 3 in 22m 24s

val Loss: 0.9863 Acc: 65.1402
Training epochs 3 in 24m 4s

Epoch 4/29
----------
train Loss: 0.6483 Acc: 73.2533
Training epochs 4 in 22m 26s

val Loss: 0.8860 Acc: 66.8224
Training epochs 4 in 24m 6s

Epoch 5/29
----------
train Loss: 0.6127 Acc: 74.7200
Training epochs 5 in 22m 26s

val Loss: 1.0138 Acc: 66.4486
Training epochs 5 in 24m 6s

Epoch 6/29
----------
train Loss: 0.5859 Acc: 76.5867
Training epochs 6 in 22m 25s

val Loss: 0.8810 Acc: 69.1589
Training epochs 6 in 24m 5